# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here


%matplotlib inline
%config InlineBackend.figure_format = 'retina'


import os
from collections import OrderedDict
from torch import nn, optim
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import json

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
#Use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [4]:
# Define your transforms for the training, validation, and testing sets
data_transforms = {
    'train' : transforms.Compose([transforms.RandomRotation(30),
                                 transforms.RandomResizedCrop(224),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(), 
                                 transforms.Normalize([0.485, 0.456, 0.406],
                                                     [0.229, 0.224, 0.225])]),
    'test' : transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406],
                                                     [0.229, 0.224, 0.225])]),
    
    'valid' :transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406],
                                                     [0.229, 0.224, 0.225])])
    
}

# Load the datasets with ImageFolder
image_datasets = {'train' : datasets.ImageFolder(train_dir, data_transforms['train']),
               'test': datasets.ImageFolder(test_dir, data_transforms['test']),
               'valid': datasets.ImageFolder(valid_dir, data_transforms['valid'])}

# Using the image datasets and the trainforms, define the dataloaders
dataloaders = {'train' : torch.utils.data.DataLoader(image_datasets['train'], 
                                                     batch_size = 32, 
                                                     shuffle=True),
               'test' : torch.utils.data.DataLoader(image_datasets['test'], 
                                                     batch_size = 32, 
                                                     shuffle=True),
               'valid': torch.utils.data.DataLoader(image_datasets['valid'], 
                                                     batch_size = 32, 
                                                     shuffle=True)}

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [5]:


with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
# import pretrained VGG

model = models.vgg16(pretrained=True)

In [7]:
# Freeze model params
for param in model.parameters():
    param.requires_grad = False

# define classifier to be used on top of VGG
n_epochs = 5
learn_rate =.002
n_classes = 102
n_inputs = 25088 # from in_features above
n_hidden = 4069
drop_p = .2
flower_classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(n_inputs,n_hidden)),
    ('relu1', nn.ReLU()),
    ('fc2', nn.Linear(n_hidden,n_classes)),
    ('dropout', nn.Dropout(drop_p)),
    ('logsoftmax', nn.LogSoftmax(dim=1))
]))

model.classifier = flower_classifier

#loss function, opimizer

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr = learn_rate)



model.to(device);

In [8]:
#training routine
running_loss = 0
steps = 0
print_every = 10
train_size = len(dataloaders['train'])
model.train()
for epoch in range(n_epochs):
    for images, flowers in dataloaders['train']:
        images, flowers = images.to(device), flowers.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(images)
        loss = criterion(logps, flowers)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        steps += 1
        
        print(f"Epoch {epoch+1}/{n_epochs}.."
              f"Train Loss: {running_loss/steps:.3f}..")
#testing
    if steps % print_every == 0:
                print(f"Epoch {epoch+1}/{n_epochs}.."
              f"Train Loss: {running_loss/steps:.3f}..")
        
        
        
        
    
    

Epoch 1/5..Train Loss: 4.620..
Epoch 1/5..Train Loss: 19.855..
Epoch 1/5..Train Loss: 24.876..
Epoch 1/5..Train Loss: 26.230..
Epoch 1/5..Train Loss: 26.585..
Epoch 1/5..Train Loss: 26.488..
Epoch 1/5..Train Loss: 24.888..
Epoch 1/5..Train Loss: 23.189..
Epoch 1/5..Train Loss: 21.921..
Epoch 1/5..Train Loss: 20.672..
Epoch 1/5..Train Loss: 19.545..
Epoch 1/5..Train Loss: 18.671..
Epoch 1/5..Train Loss: 17.789..
Epoch 1/5..Train Loss: 16.957..
Epoch 1/5..Train Loss: 16.200..
Epoch 1/5..Train Loss: 15.487..
Epoch 1/5..Train Loss: 14.870..
Epoch 1/5..Train Loss: 14.317..
Epoch 1/5..Train Loss: 13.805..
Epoch 1/5..Train Loss: 13.330..
Epoch 1/5..Train Loss: 12.912..
Epoch 1/5..Train Loss: 12.525..
Epoch 1/5..Train Loss: 12.175..
Epoch 1/5..Train Loss: 11.826..
Epoch 1/5..Train Loss: 11.514..
Epoch 1/5..Train Loss: 11.234..
Epoch 1/5..Train Loss: 10.967..
Epoch 1/5..Train Loss: 10.707..
Epoch 1/5..Train Loss: 10.480..
Epoch 1/5..Train Loss: 10.272..
Epoch 1/5..Train Loss: 10.085..
Epoch 1/5

Epoch 2/5..Train Loss: 3.621..
Epoch 2/5..Train Loss: 3.614..
Epoch 2/5..Train Loss: 3.614..
Epoch 2/5..Train Loss: 3.609..
Epoch 2/5..Train Loss: 3.603..
Epoch 2/5..Train Loss: 3.597..
Epoch 2/5..Train Loss: 3.594..
Epoch 2/5..Train Loss: 3.589..
Epoch 2/5..Train Loss: 3.582..
Epoch 2/5..Train Loss: 3.576..
Epoch 2/5..Train Loss: 3.573..
Epoch 2/5..Train Loss: 3.569..
Epoch 2/5..Train Loss: 3.564..
Epoch 2/5..Train Loss: 3.560..
Epoch 2/5..Train Loss: 3.555..
Epoch 2/5..Train Loss: 3.553..
Epoch 2/5..Train Loss: 3.548..
Epoch 2/5..Train Loss: 3.541..
Epoch 2/5..Train Loss: 3.537..
Epoch 2/5..Train Loss: 3.533..
Epoch 2/5..Train Loss: 3.529..
Epoch 2/5..Train Loss: 3.523..
Epoch 2/5..Train Loss: 3.519..
Epoch 2/5..Train Loss: 3.514..
Epoch 2/5..Train Loss: 3.508..
Epoch 2/5..Train Loss: 3.505..
Epoch 2/5..Train Loss: 3.502..
Epoch 2/5..Train Loss: 3.498..
Epoch 2/5..Train Loss: 3.493..
Epoch 2/5..Train Loss: 3.489..
Epoch 2/5..Train Loss: 3.485..
Epoch 2/5..Train Loss: 3.482..
Epoch 2/

Epoch 3/5..Train Loss: 2.924..
Epoch 3/5..Train Loss: 2.922..
Epoch 3/5..Train Loss: 2.921..
Epoch 3/5..Train Loss: 2.920..
Epoch 3/5..Train Loss: 2.918..
Epoch 3/5..Train Loss: 2.916..
Epoch 3/5..Train Loss: 2.914..
Epoch 3/5..Train Loss: 2.911..
Epoch 3/5..Train Loss: 2.909..
Epoch 3/5..Train Loss: 2.908..
Epoch 3/5..Train Loss: 2.907..
Epoch 3/5..Train Loss: 2.907..
Epoch 3/5..Train Loss: 2.905..
Epoch 3/5..Train Loss: 2.903..
Epoch 3/5..Train Loss: 2.902..
Epoch 3/5..Train Loss: 2.900..
Epoch 3/5..Train Loss: 2.900..
Epoch 3/5..Train Loss: 2.898..
Epoch 3/5..Train Loss: 2.896..
Epoch 3/5..Train Loss: 2.896..
Epoch 3/5..Train Loss: 2.895..
Epoch 3/5..Train Loss: 2.893..
Epoch 3/5..Train Loss: 2.893..
Epoch 3/5..Train Loss: 2.892..
Epoch 3/5..Train Loss: 2.891..
Epoch 3/5..Train Loss: 2.891..
Epoch 3/5..Train Loss: 2.889..
Epoch 3/5..Train Loss: 2.888..
Epoch 3/5..Train Loss: 2.887..
Epoch 3/5..Train Loss: 2.886..
Epoch 3/5..Train Loss: 2.884..
Epoch 3/5..Train Loss: 2.882..
Epoch 3/

Epoch 4/5..Train Loss: 2.628..
Epoch 4/5..Train Loss: 2.627..
Epoch 4/5..Train Loss: 2.625..
Epoch 4/5..Train Loss: 2.623..
Epoch 4/5..Train Loss: 2.621..
Epoch 4/5..Train Loss: 2.620..
Epoch 4/5..Train Loss: 2.620..
Epoch 4/5..Train Loss: 2.618..
Epoch 4/5..Train Loss: 2.617..
Epoch 4/5..Train Loss: 2.617..
Epoch 4/5..Train Loss: 2.616..
Epoch 4/5..Train Loss: 2.616..
Epoch 4/5..Train Loss: 2.615..
Epoch 4/5..Train Loss: 2.614..
Epoch 4/5..Train Loss: 2.613..
Epoch 4/5..Train Loss: 2.612..
Epoch 4/5..Train Loss: 2.611..
Epoch 4/5..Train Loss: 2.610..
Epoch 4/5..Train Loss: 2.611..
Epoch 4/5..Train Loss: 2.611..
Epoch 4/5..Train Loss: 2.609..
Epoch 4/5..Train Loss: 2.608..
Epoch 4/5..Train Loss: 2.608..
Epoch 4/5..Train Loss: 2.607..
Epoch 4/5..Train Loss: 2.606..
Epoch 4/5..Train Loss: 2.605..
Epoch 4/5..Train Loss: 2.604..
Epoch 4/5..Train Loss: 2.604..
Epoch 5/5..Train Loss: 2.603..
Epoch 5/5..Train Loss: 2.603..
Epoch 5/5..Train Loss: 2.601..
Epoch 5/5..Train Loss: 2.600..
Epoch 5/

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [9]:
# TODO: Do validation on the test set
model.eval()

test_loss = 0
accuracy = 0
model.eval()
with torch.no_grad():
    test_size = len(dataloaders["test"])
    for images, flowers in dataloaders['test']:

        images, flowers = images.to(device), flowers.to(device)
        logps = model.forward(images)
        batch_loss = criterion(logps, flowers)
        test_loss += batch_loss.item()

        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim =1)
        is_match = top_class == flowers.view(*top_class.shape)
        accuracy += torch.mean(is_match.type(torch.cuda.FloatTensor)).item()


    print(f"Test Loss: {test_loss/train_size:.3f}.."
         f"Avg. Test accuracy: {accuracy/test_size:.3f}")
    model.train()

                

Test Loss: 0.143..Avg. Test accuracy: 0.737


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [28]:
# Save the checkpoint and default params
model.class_to_idx =cat_to_name
checkpoint = {
    'input_size' : n_inputs,
    'output_size' : n_classes,
    'hidden_size' : n_hidden,
    'state_dict' : model.classifier.state_dict(),
    'epochs': n_epochs,
    'learn_rate': learn_rate,
    'optimizer' : optimizer,
    'optim_state': optimizer.state_dict(),
    'dropout_prob': drop_p,
    'criterion' : loss,
    'class_to_idx': model.class_to_idx
}



torch.save(checkpoint, 'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [35]:
def build_model_from_file(filename):
    checkpoint = torch.load(filename)
    

    
    class_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(checkpoint['input_size'],checkpoint['hidden_size'],)),
    ('relu1', nn.ReLU()),
    ('fc2', nn.Linear(checkpoint['hidden_size'],checkpoint['output_size'])),
    ('dropout', nn.Dropout(checkpoint['dropout_prob'])),
    ('logsoftmax', nn.LogSoftmax(dim=1))
]))
    
    class_model.load_state_dict(checkpoint['state_dict'])
    base_model.classifier = class_model
    loss = checkpoint['criterion']
    optimizer = checkpoint['optimizer'].load_state_dict(checkpoint['optim_state'])
    base_model.class_to_idx = checkpoint['class_to_idx']
    return[base_model, loss, optimizer]

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [20]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    im = Image.open(image)
    
    min_size = min(im.size)

    if im.size[0] == min_size:
        new_size = [im.size[0], 256]
    else:
        new_size = [256, im.size[1]]
    
    im.thumbnail(new_size)
    width, height = im.size  

    left = (256 - 224)/2
    top = (256 - 224)/2
    right = (256 + 224)/2
    bottom = (256 + 224)/2

    image = im.crop((left, top, right, bottom))
    
    im = np.array(im)
    im = im/255.
    
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    im = (im - mean) / std
    
    im = np.transpose(im, (2, 0, 1))
    
    return im

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [21]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [120]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    
       
    model.to(device)
    
    model.eval()
    
    img_data = process_image(image_path)
    in_tensor = torch.from_numpy(img_data)
    in_tensor = in_tensor.type(torch.FloatTensor)
    
    images = in_tensor.to(device)
    images.unsqueeze_(0)
    
    output = model.forward(images)
    ps = torch.exp(output)
    
    class_dict = model.class_to_idx
    
    inv_dict =  class_dict#{v: k for k,v in class_dict.items()}
    
    top_p, top_classes = ps.topk(5, dim =1) 
    print([inv_dict[str(clid)] for clid in top_classes.tolist()[0]])
    print(top_p.tolist()[0])

    return top_p, top_classes
    
    
    # TODO: Implement the code to predict the class from an image file
    

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [121]:
predict('flowers/test/28/image_05230.jpg',model)

['stemless gentian', 'camellia', 'ruby-lipped cattleya', 'alpine sea holly', 'morning glory']
[0.470257043838501, 0.3054981231689453, 0.11604566872119904, 0.02462361939251423, 0.02343548834323883]


(tensor([[0.4703, 0.3055, 0.1160, 0.0246, 0.0234]], device='cuda:0',
        grad_fn=<TopkBackward>),
 tensor([[28, 96, 36, 35, 76]], device='cuda:0'))